<a href="https://colab.research.google.com/github/DaisyLaw/Machine-Learning-practices/blob/main/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet tf_keras

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import tf_keras

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)

# Check for GPU
print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

train_df = pd.read_csv("/content/sample_data/Digit_Recognizer/Data/train.csv")
train_df.head()

In [ ]:
len(train_df)

In [ ]:
train_df['label'].unique()

In [ ]:
plt.hist(train_df['label'], bins=range(11), rwidth=0.6, align='left')
plt.xticks(range(10))
plt.title("Distribution of labels across training data")
plt.ylabel('Occurences')
plt.xlabel('Labels')
plt.show()

In [ ]:
X = train_df.iloc[:,1:]
X = X.values.reshape(-1, 28, 28, 1)
X = X/255.0
X.shape

In [ ]:
plt.figure(figsize=(6,6))
for num in range(0,25):
    plt.subplot(5,5,num+1)
    plt.imshow(X[num, :, :, :], interpolation = "none", cmap = plt.cm.binary)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
y = train_df["label"]
len(y)

In [ ]:
# Import train_test_split from Scikit-Learn
from sklearn.model_selection import train_test_split

#Splitting the training data into traizning and validation data for the first 1000 digits.
X_train, X_validation, y_train, y_validation = train_test_split(X,
                                                                y,
                                                                test_size = 0.2,
                                                                random_state = 10)

len(X_train), len(y_train), len(X_validation), len(y_validation)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=20,  # randomly rotate images in the range 20 degrees
        zoom_range = 0.2, # Randomly zoom image 20%
        width_shift_range=0.2,  # randomly shift images horizontally 20%
        height_shift_range=0.2,  # randomly shift images vertically 20%
        shear_range=0.2) #shear intensity 20%


datagen.fit(X_train)

In [ ]:
#Define an accuracy callback
class myAccuracyCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('val_accuracy') is not None and logs.get('val_accuracy') > 0.999:
                print("\nReached 99.9% accuracy so cancelling training!")

                # Stop training once the above condition is met
                self.model.stop_training = True

In [ ]:
#Initialize hyperparameters
EPOCHS = 50
BATCH_SIZE = 250

# Instantiate the callback class
accuracyCallbacks = myAccuracyCallback()
#tensorboard_callback = tf.keras.callbacks.TensorBoard()

# Create early stopping (once our model stops improving, stop training)
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
#                                                  patience=3) # stops after 3 rounds of no improvements

# Define the model
model = tf_keras.models.Sequential([
    tf_keras.layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=[28, 28, 1]),
    tf_keras.layers.MaxPooling2D(2),
    tf_keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    tf_keras.layers.MaxPooling2D(2),
    tf_keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf_keras.layers.MaxPooling2D(2),
    tf_keras.layers.Flatten(),
    tf_keras.layers.Dense(512, activation='relu'),
    tf_keras.layers.Dropout(rate=0.3),
    tf_keras.layers.Dense(256, activation='relu'),
    tf_keras.layers.Dropout(rate=0.3),
    tf_keras.layers.Dense(128, activation='relu'),
    tf_keras.layers.Dropout(rate=0.3),
    tf_keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model adding the callback and save the training history
history_1000_data = model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                    validation_data=(X_validation, y_validation),
                    steps_per_epoch=len(X_train)/BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[accuracyCallbacks])

In [ ]:
#Summarize history for accuracy
plt.plot(history_1000_data.history['accuracy'])
plt.plot(history_1000_data.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.plot(history_1000_data.history['loss'])
plt.plot(history_1000_data.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X,
                                                                y,
                                                                test_size = 0.2,
                                                                random_state = 10)

len(X_train), len(y_train), len(X_validation), len(y_validation)

In [ ]:
history_full_data = model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                              validation_data=(X_validation, y_validation),
                              steps_per_epoch=len(X_train)/BATCH_SIZE,
                              epochs=EPOCHS,
                              callbacks=[accuracyCallbacks])

In [ ]:
#Summarize history for accuracy
plt.plot(history_full_data.history['accuracy'])
plt.plot(history_full_data.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.plot(history_full_data.history['loss'])
plt.plot(history_full_data.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
path_test = "/content/sample_data/Digit_Recognizer/Data/test.csv"
test_df = pd.read_csv(path_test)
test_df.info()

In [ ]:
test_df.head()

In [ ]:
X_test = test_df.values.reshape(-1, 28, 28, 1)
X_test = X_test/255.0
X_test.shape

In [ ]:
plt.figure(figsize=(6,6))
for num in range(0,25):
    plt.subplot(5,5,num+1)
    plt.imshow(X_test[num, :, :, :], interpolation = "none", cmap = plt.cm.binary)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
y_pred = model.predict(X_test)
preds_test = np.argmax(y_pred,axis = 1)

sample_sub = pd.read_csv("/content/sample_data/Digit_Recognizer/sample_submission.csv")
test_id = sample_sub.ImageId.copy()

results = pd.Series(preds_test,name="Label")
output = pd.DataFrame({'ImageId': test_id,
                       'Label': preds_test})

output.head(10)

In [ ]:
output.to_csv('/content/sample_data/Digit_Recognizer/submission-01.csv', index=False)